In [1102]:
import nltk
from nltk.util import ngrams
import string
import json
from collections import Counter
import random
import re
 
# "..." -> {...}
def extract_ngrams(data, num):
    n_grams = ngrams(data.split(' '), num)
    return [ ' '.join(grams) for grams in n_grams]

# remove urls
def clean_text(txt):
    # remove urls
    txt = re.sub(r"https://[A-Za-z]+.[A-Za-z]*", '',txt)
#     txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

# generates a sentence seed
def startsent():
    beg = []
    for b in bigrams:
        first = b.split(' ')[0]
        # TO DO: CHANGE CHOOSE CONDITION DYNAMICALLY BASED ON FILE SIZE
        if (first == "begsent" and bigrams[b] > 1):
            beg.append(b)
            
    return random.choice(beg)

# gets the next word given the previous
# random choice from 10 most popular ngrams
def getword(word):
    possiblewords = []
    newword=""
    
    for b in bigrams:
        first = b.split(' ')[0]
        
        # don't want to add too irrelevant bigrams to choose from
        if(len(possiblewords) > 10): break;
        # TO DO: CHANGE CHOOSE CONDITION DYNAMICALLY BASED ON FILE SIZE
        if (first == word and bigrams[b] > 0):
            newword = b.split(' ')[1]
            possiblewords.append(newword)

    if (len(possiblewords) == 0):
        return ""
    else:
        newword = random.choice(possiblewords)
        if(newword == "endsent"): return ""
        else: return newword
    

def word_gen(bigrams):
    seed = startsent()
    sent = seed.split(' ')[1]
    
    nextword = getword(sent)
    while (nextword != "" and len(sent) < 140):
        sent += " " + nextword
        nextword = getword(nextword)
        
    # if too many @'s, retry
    if(len(re.findall('@', sent)) == len(sent.split(' '))):
       sent = word_gen(bigrams)
    
    return sent
    
    

In [1059]:
### takin an n grams approach ###
def writengrams(city):
    with open("cities/" + city, "r") as content:
        data = json.load(content)

    all_tweets = []
    for tweet in data:
        updatetweet = "begsent " + tweet['tweet'][0]['text'] + " endsent"
        all_tweets.append(updatetweet)

    all_tweets = [h for h in all_tweets if h != "Unknown"]

    corpus = [clean_text(x) for x in all_tweets]

    bigrams = {}

    for tweet in corpus:
        bigram = extract_ngrams(tweet.lower(), 2)
        for b in bigram:
            if b in bigrams:
                bigrams[b] +=1
            else:
                bigrams[b] = 1
            
    sortedbigrams = {k: v for k, v in sorted(bigrams.items(), key=lambda item: item[1], reverse=True)}
    
    with open("ngramsbycity/" + city, 'w') as file:
        file.write(json.dumps(sortedbigrams))
    
    
    
# print(Counter(allngs))


In [ ]:
cityfile = open('writtencities', r)
cities = json.load(cityfile)

for f in cities:
    city = re.sub(" ", "", f.split(',')[0].lower())
    writengrams(city)

In [1127]:
print(word_gen(sortedbigrams))

just hit 3k in a sin… /gg73aoexno
